In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
folder_path ="D:\phd stuff\\tidy3d\structures\\06_05_2024"
project_name = "LSU Tight Beam"
postprocess_results = []
runtime = 500
min_steps_per_lambda = 30
for direction in ["z"]: 
    for filename in os.listdir(folder_path):
     for cutting in [1.0]:
        print(filename)
        if not (Path(filename).suffix==".h5"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=[8.0,6.0],
                                            box_size=14.3,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda,
                                           scaling=1,shuoff_condtion=1e-12, verbose=True,
                                             #monitors=["flux","field_monitor","permittivity_monitor","field_time_domain","time_monitor"], 
                                             monitors=["flux","time_monitor"], 
                                           freqs=400, 
                                           cut_condition=cutting, source="tight")
            
           
            print(structure_1.__str__())
            structure_1.plot_sim_layout()
            #print(structure_1.estimate_cost())
            results = structure_1.run_sim(folder_description=project_name,load=False,run_free=False, add_ref=True, run=False)
          


            


In [ ]:
folder_path = "/Users/francisco/phd stuff/tidy3d/structures/test_stl_17_04_2024"
project_name = "Convergence_STL_Test_phi_Absorption"
#index_dist = "https://refractiveindex.info/data_csv.php?datafile=database/data-nk/main/Si/Aspnes.yml" #0.21 - 0.83
#index_dist = "https://refractiveindex.info/data_csv.php?datafile=database/data-nk/main/Si/Schinke.yml" #0.25 - 1.45
index_dist = "https://web.archive.org/web/20230207133303/https://refractiveindex.info/data_csv.php?datafile=data/main/Si/Schinke.yml" #From archive, refractiveindex.info was down

runtime = 1000
min_steps_per_lambda = 50
for direction in ["z"]: 
    for filename in os.listdir(folder_path):
     print(filename)
     #for perm in [11.56,9.0,7.5,6.25,3.6]:
     for scale in [0.8,0.85,1.0,1.25]:
        print("scale:",scale)
        if not (Path(filename).suffix==".stl"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=[1.3,0.5],
                                            box_size=(14.3/10),runtime=runtime,min_steps_per_lambda=min_steps_per_lambda, permittivity=11.56, permittivity_dist= index_dist,
                                           scaling=scale,shuoff_condtion=1e-8)
            
            # while structure_1.sim.num_cells >= 1e8:
            #       print("cells:",structure_1.sim.num_cells)
            #       min_steps_per_lambda -= 1
            #       structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
            #                                 ,direction=direction, lambda_range=[1.1,0.9],
            #                                 box_size=1.7875,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda, permittivity=11.68, 
            #                                scaling=1/scale)
            
            
            # while structure_1.sim.num_time_steps >= 50000:
            #       print("steps:",structure_1.sim.num_time_steps)
            #       runtime -= 5
            #       structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
            #                                 ,direction=direction, lambda_range=[1.1,0.9],
            #                                 box_size=1.7875,runtime=runtime,min_steps_per_lambda=min_steps_per_lambda, permittivity=11.68, 
            #     
            #                            scaling=1/scale)
            print(structure_1.__str__())
            structure_1.plot_sim_layout()
            #print(structure_1.estimate_cost())
            
            results = structure_1.run_sim(folder_description=project_name,load=True,run_free=False)

            sim_data, sim_data0, task_name_def = results
            #Plot flux
            #Retrieve the power flux through the monitor plane.
            transmission0 = sim_data0['flux1'].flux
            transmission = sim_data['flux1'].flux
            transmission_normalized = transmission / transmission0
            transmission_flux_right = sim_data['flux1'].flux
            transmission_flux_left = sim_data['flux2'].flux
            monitor_lambdas = td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)
            #plt.semilogy(monitor_lambdas, (transmission_flux_left/transmission0), label='Left Monitor')
            plt.semilogy(monitor_lambdas, (transmission_flux_right/transmission0), label='Right Monitor')
            plt.xlabel('$\lambda (\mu m)$')
            plt.ylabel('T')
            plt.legend()
            plt.grid()
            plt.title(f"{task_name_def}\n")
            plt.show()
    ##

            
